In [1]:
import numpy as np
import pandas as pd
import xlrd
import json
import os
from flask import Flask, redirect, url_for, request, render_template, session
import plotly.express as px
import plotly.graph_objs as go
import geopandas as gpd
import pickle
import pyproj

In [ ]:
app = Flask(__name__)
app.secret_key = 'aqswdefrgt'
TEMPLATES_AUTO_RELOAD = True

@app.route("/readData",methods = ["POST","GET"])
def readData():
    data = {}
    data["Test"] = "Working"
    return json.dumps(data,indent=2)


if __name__ == '__main__':
    app.run(host="0.0.0.0")